In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import os

In [2]:
#Path of the file with the data of particles
os.chdir('/home/user/Documents/BUAP/Estancia/CosmicRayReco/MCAnalysis/ClusterAnalysis/Data')

In [17]:
#File with summary of Showers
Showers = pd.read_csv('DAT000001_showers.txt',delimiter='\t')

In [18]:
#File with information of all particles at observation level
Particles = pd.read_csv('DAT000001_particles.txt',delimiter='\t')

In [19]:
#Change the units of first interaction point from cm to km
Showers['Zfirst'] = Showers['Zfirst'].mul(1e-5)

KeyError: 'Zfirst'

In [ ]:
#change units of positions from cm to m
Particles[['x','y']] = Particles[['x','y']].mul(0.01)

In [ ]:
#Statistics of the data frame with summary of showers 
Showers.describe()

In [20]:
#Definition of a box which contain all the particles of the simulation
#size of the detector on mm (¡¡¡For the moment work with a detector of 1m^2 !!!)

det_s_x = 1 # detector size in x axis (m)
det_s_y = 1 # detector size in y axis (m)


In [23]:
#Add a new column which will contain the number of cluster for each particle
Particles["DetPosx"] = np.ceil(Particles["x (m)"])
Particles["DetPosy"] = np.ceil(Particles["y (m)"]) 
Particles["CoorDetx"] = (Particles["x (m)"]) % det_s_x
Particles["CoorDety"] = (Particles["y (m)"]) % det_s_y
Particles['Ncluster'] = 0

In [24]:
#Statistics of the data frame of all particles at observation level
Particles.describe()

,N. sh,P. id,x (m),y (m),t (ns),px (GeV/c),py (GeV/c),pz (GeV/c),Psq (GeV),ek (GeV),zha (°),aza (°),DetPosx,DetPosy,CoorDetx,CoorDety,Ncluster
count,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.000000,6529.0
mean,5011.253484,4.896309,16.978040,-51.585109,58998.878170,0.016764,-0.013843,1.733447,11.155902,1.714687,19.117680,-1.966183,17.480778,-51.083933,0.493280,0.495761,0.0
std,2922.483435,3.325776,1294.230225,1336.727117,28066.028237,0.725566,0.712764,2.667839,51.320444,2.770304,9.927801,103.483712,1294.223425,1336.732924,0.288855,0.286825,0.0
min,2.000000,1.000000,-10000.600000,-9878.090000,7681.560000,-12.906700,-8.695240,0.028098,0.002507,0.050068,0.318068,-179.971000,-10000.000000,-9878.000000,0.000000,0.000000,0.0
25%,2423.000000,2.000000,-565.495000,-607.096000,39672.900000,-0.118768,-0.134109,0.175024,0.037130,0.168896,12.270700,-89.082500,-565.000000,-607.000000,0.249000,0.250000,0.0
50%,4976.000000,5.000000,23.574400,-24.180800,54937.000000,0.001771,-0.002852,0.850899,0.867927,0.697298,18.107100,-5.900790,24.000000,-24.000000,0.490000,0.496000,0.0
75%,7586.000000,6.000000,575.632000,535.185000,74649.100000,0.130324,0.112526,2.110490,5.023990,2.061770,24.269100,87.388900,576.000000,536.000000,0.744000,0.742000,0.0
max,9992.000000,14.000000,11060.300000,11834.200000,261210.000000,8.690300,10.360700,35.362900,1313.890000,36.142100,84.229700,179.997000,11061.000000,11835.000000,0.998700,0.999820,0.0


In [35]:
#Define a function to find the clusters of particles
def Clusters(ith_clust, Sh_n):
    
    # Transform the columns of the particle data frame with the position of the detector into a list
    x_pos = Sh_n['DetPosx'].tolist()
    y_pos = Sh_n['DetPosy'].tolist()
    
    # Create an empty set
    unique_pairs = set()

    # Iterate over the pairs of x and y using zip
    for xi, yi in zip(x_pos, y_pos):
        pair = (xi, yi)
        
        if(xi==410.0 and yi==1899.0):
            print(pair)

        # Check if the pair is already in the set
        if pair not in unique_pairs:
            # If not, add it to the set
            unique_pairs.add(pair)

            # Extract the particles index by comparing it with the x, y position in the data frame
            p_index_clust = Sh_n.index[ (Sh_n['DetPosx'] == xi) & (Sh_n['DetPosy'] == yi) ].tolist()
            
            # Replacte the cluster position of the particle for the cluster id
            for j in range(len(p_index_clust)):
                Sh_n.at[p_index_clust[j],'Ncluster'] = ith_clust

            ith_clust = ith_clust+1 # Next Cluster
    
    return Sh_n, ith_clust 
            

In [48]:
#Define a function to find the clusters of particles
def Clusters_v2(ith_clust, Sh_n):
    
    # Transform the columns of the particle data frame with the position of the detector into a list
    x_pos = Sh_n['DetPosx'].tolist()
    y_pos = Sh_n['DetPosy'].tolist()
    
    # Create an empty set
    unique_pairs = set()

    # Iterate over the pairs of x and y using zip
    for xi in x_pos:
        for yi in y_pos:
            pair = (xi, yi)

            if(xi==410.0 and yi==1899.0):
                print(pair)

            # Check if the pair is already in the set
            if pair not in unique_pairs:
                # If not, add it to the set
                unique_pairs.add(pair)

                # Extract the particles index by comparing it with the x, y position in the data frame
                p_index_clust = Sh_n.index[ (Sh_n['DetPosx'] == xi) & (Sh_n['DetPosy'] == yi) ].tolist()

                # Replacte the cluster position of the particle for the cluster id
                for j in range(len(p_index_clust)):
                    Sh_n.at[p_index_clust[j],'Ncluster'] = ith_clust

                ith_clust = ith_clust+1 # Next Cluster
    
    return Sh_n, ith_clust 

In [46]:
a = [1,2,3]
b = [2,3,4]
for ai in a:
    for bi in b:
        pair=(ai,bi)
        print(pair)

(1, 2)
(1, 3)
(1, 4)
(2, 2)
(2, 3)
(2, 4)
(3, 2)
(3, 3)
(3, 4)


In [55]:
Showers["N. Shower"].tolist()

[2,
 5,
 8,
 11,
 13,
 14,
 15,
 17,
 18,
 20,
 24,
 34,
 47,
 49,
 50,
 51,
 52,
 56,
 58,
 60,
 64,
 73,
 75,
 76,
 83,
 84,
 90,
 100,
 104,
 105,
 108,
 110,
 114,
 115,
 117,
 118,
 119,
 120,
 121,
 122,
 125,
 128,
 129,
 131,
 132,
 134,
 136,
 139,
 141,
 142,
 143,
 146,
 152,
 159,
 164,
 165,
 167,
 171,
 174,
 175,
 181,
 186,
 188,
 193,
 196,
 200,
 205,
 207,
 210,
 214,
 215,
 216,
 217,
 223,
 224,
 230,
 234,
 235,
 237,
 242,
 244,
 245,
 246,
 251,
 253,
 262,
 264,
 265,
 276,
 278,
 281,
 285,
 287,
 289,
 292,
 294,
 297,
 301,
 310,
 311,
 314,
 317,
 320,
 322,
 323,
 324,
 326,
 328,
 330,
 334,
 335,
 339,
 342,
 343,
 346,
 363,
 365,
 369,
 377,
 378,
 380,
 383,
 386,
 387,
 390,
 391,
 393,
 397,
 405,
 414,
 415,
 423,
 424,
 425,
 427,
 428,
 431,
 433,
 434,
 441,
 442,
 446,
 447,
 448,
 451,
 452,
 454,
 455,
 458,
 462,
 463,
 464,
 471,
 473,
 474,
 475,
 476,
 482,
 484,
 485,
 491,
 492,
 493,
 500,
 503,
 508,
 513,
 514,
 515,
 519,
 522,
 529

In [56]:
%%time
ith_clust = 1 # First cluster

# Run over all the showers
for ith_shower in (Showers["N. Shower"].tolist()):
    Sh_n, ith_clust = Clusters_v2(ith_clust, Particles[Particles["N. sh"]==ith_shower]) 
    Particles.loc[Particles["N. sh"]==ith_shower, :] = Sh_n

(410.0, 1899.0)
CPU times: user 44.2 s, sys: 32 ms, total: 44.3 s
Wall time: 44.3 s


In [57]:
# Check if there are particles that are not counted
Particles[Particles["Ncluster"]==0]

,N. sh,P. id,x (m),y (m),t (ns),px (GeV/c),py (GeV/c),pz (GeV/c),Psq (GeV),ek (GeV),zha (°),aza (°),DetPosx,DetPosy,CoorDetx,CoorDety,Ncluster


In [58]:
# Number of the cluster with the maximum number of particles
df_sn = Particles[Particles["Ncluster"]!=0]
print(df_sn["Ncluster"].value_counts().idxmax())

19530


In [60]:
# Summary of the cluster with the maximum number of particles
Particles[Particles["Ncluster"]==19530].describe()

,N. sh,P. id,x (m),y (m),t (ns),px (GeV/c),py (GeV/c),pz (GeV/c),Psq (GeV),ek (GeV),zha (°),aza (°),DetPosx,DetPosy,CoorDetx,CoorDety,Ncluster
count,4.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.0,4.0,4.000000,4.000000,4.0
mean,7556.0,1.750000,-1006.490000,1931.350000,55955.100000,-0.100733,0.035492,0.339599,0.196370,0.355748,17.349425,160.691000,-1006.0,1932.0,0.510000,0.350000,19530.0
std,0.0,0.957427,0.110151,0.198494,0.141421,0.087137,0.031560,0.290275,0.293296,0.304502,0.228002,1.165675,0.0,0.0,0.110151,0.198494,0.0
min,7556.0,1.000000,-1006.620000,1931.060000,55954.900000,-0.226749,0.009392,0.089501,0.008782,0.093715,17.090100,159.852000,-1006.0,1932.0,0.380000,0.060000,19530.0
25%,7556.0,1.000000,-1006.560000,1931.307500,55955.050000,-0.114874,0.020787,0.210627,0.053914,0.220378,17.208225,160.146750,-1006.0,1932.0,0.440000,0.307500,19530.0
50%,7556.0,1.500000,-1006.480000,1931.420000,55955.150000,-0.075016,0.025583,0.254962,0.071311,0.266749,17.348300,160.247500,-1006.0,1932.0,0.520000,0.420000,19530.0
75%,7556.0,2.250000,-1006.410000,1931.462500,55955.200000,-0.060875,0.040289,0.383934,0.213767,0.402120,17.489500,160.791750,-1006.0,1932.0,0.590000,0.462500,19530.0
max,7556.0,3.000000,-1006.380000,1931.500000,55955.200000,-0.026151,0.081410,0.758969,0.634077,0.795779,17.611000,162.417000,-1006.0,1932.0,0.620000,0.500000,19530.0


In [61]:
Particles[Particles["Ncluster"]==19530]

,N. sh,P. id,x (m),y (m),t (ns),px (GeV/c),py (GeV/c),pz (GeV/c),Psq (GeV),ek (GeV),zha (°),aza (°),DetPosx,DetPosy,CoorDetx,CoorDety,Ncluster
4860,7556,1,-1006.62,1931.39,55955.2,-0.026151,0.009392,0.089501,0.008782,0.093715,17.2476,160.245,-1006.0,1932.0,0.38,0.39,19530
4861,7556,1,-1006.42,1931.45,55955.1,-0.072449,0.026581,0.251003,0.068958,0.262599,17.0901,159.852,-1006.0,1932.0,0.58,0.45,19530
4862,7556,3,-1006.38,1931.06,55954.9,-0.077582,0.024585,0.258922,0.073664,0.270900,17.4490,162.417,-1006.0,1932.0,0.62,0.06,19530
4863,7556,2,-1006.54,1931.50,55955.2,-0.226749,0.081410,0.758969,0.634077,0.795779,17.6110,160.250,-1006.0,1932.0,0.46,0.50,19530
